In [ ]:
#import libraries
#! pip install kaggle
#create token from kaggle settings and place it in home directory under .Kaggle folder.

In [9]:
#Downloading dataset from Kaggle
import kaggle
! kaggle datasets download ankitbansal06/retail-orders -f orders.csv  
#(-f orders.csv (to get specific file orders.csv from that page))

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0
orders.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [10]:
#extract file from zip file
import zipfile
zipped=zipfile.ZipFile("orders.csv.zip")
zipped.extractall()
zipped.close()

In [67]:
#Loading file into pandas dataframe
import pandas as pd
df=pd.read_csv("orders.csv",encoding="unicode_escape")
df.head()

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [68]:
df["Ship Mode"].unique()  
#column values ('Not Available', 'unknown') needs to be replaced with nan.

array(['Second Class', 'Standard Class', 'Not Available', 'unknown',
       'First Class', nan, 'Same Day'], dtype=object)

In [69]:
import numpy as np
df["Ship Mode"].replace(('Not Available', 'unknown',),np.nan,inplace=True)
df["Ship Mode"].unique() 

C:\Users\rohitzch\AppData\Local\Temp\ipykernel_30544\61841934.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Ship Mode"].replace(('Not Available', 'unknown',),np.nan,inplace=True)


array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [70]:
df.columns

Index(['Order Id', 'Order Date', 'Ship Mode', 'Segment', 'Country', 'City',
       'State', 'Postal Code', 'Region', 'Category', 'Sub Category',
       'Product Id', 'cost price', 'List Price', 'Quantity',
       'Discount Percent'],
      dtype='object')

In [71]:
#column names should be in lower case and instead of space we have to give "_"
#df.rename(columns={'Order Id':'order_id', 'City':'city'}) By this way we have to specify all 16 columns

df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace(" ","_")
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'cost_price', 'list_price', 'quantity',
       'discount_percent'],
      dtype='object')

In [72]:
#derive new columns discount , sale price and profit

df["discount"]=df["list_price"]*df["discount_percent"]*0.01
df["sale_price"]=df["list_price"]-df["discount"]
df["profit"]=df["sale_price"]-df["cost_price"]

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          9994 non-null   int64  
 1   order_date        9994 non-null   object 
 2   ship_mode         9988 non-null   object 
 3   segment           9994 non-null   object 
 4   country           9994 non-null   object 
 5   city              9994 non-null   object 
 6   state             9994 non-null   object 
 7   postal_code       9994 non-null   int64  
 8   region            9994 non-null   object 
 9   category          9994 non-null   object 
 10  sub_category      9994 non-null   object 
 11  product_id        9994 non-null   object 
 12  cost_price        9994 non-null   int64  
 13  list_price        9994 non-null   int64  
 14  quantity          9994 non-null   int64  
 15  discount_percent  9994 non-null   int64  
 16  discount          9994 non-null   float64


In [74]:
#order date datatype is object needs to change it to datetime.

df["order_date"]=pd.to_datetime(df["order_date"],format="%Y-%m-%d") #Y for 4 digit (2024), y for 2 digit (24)

In [75]:
#drop cost price list price and discount percent columns

df.drop(columns=["cost_price","list_price","discount_percent"],inplace=True)

In [24]:
#import data into SQL
#! pip install pyodbc
#! pip install sqlalchemy

In [76]:
#creating a connection between SQL & Python SQLALCHEMY

from sqlalchemy import create_engine
import pyodbc

Server= ".\SQLEXPRESS"
Database="python_project"
Driver="ODBC+Driver+17+for+SQL+Server"
username="sa"
password="Rohit%40123" #used %40 inplace of @. it's a URL-encoding, without this connection is breaking due to @ in password.

connection_string=f"mssql+pyodbc://{username}:{password}@{Server}/{Database}?driver={Driver}"

<>:6: SyntaxWarning: invalid escape sequence '\S'
<>:6: SyntaxWarning: invalid escape sequence '\S'
C:\Users\rohitzch\AppData\Local\Temp\ipykernel_30544\3315325257.py:6: SyntaxWarning: invalid escape sequence '\S'
  Server= ".\SQLEXPRESS"


In [77]:
engine=create_engine(connection_string)

In [44]:
#For running SQL Queries in Python.
'''query="select * from df_orders" 
df=pd.read_sql(query,engine)     
df'''

'query="select * from pizza_sales" \ndf=pd.read_sql(query,engine)     \ndf'

In [79]:
#Loading data into SQL using append.

df.to_sql('df_orders', con=engine , index=False, if_exists = 'append')

38